# Retail: Sales & Product Analysis 📊

## Context and objectives

### As a consulting firm, in this projects we will be analysis the current state of the sales of a retail chain across multiple sites, products and months. The goal is to provide valuable insights in order to make crucial business decisions. 

### The company is going through a bad financial situation and needs to cut costs somehow. They have asked us to make some decisive decision for the company to move forward.


#### Metrics to analyze:

- profit and profit margins from the cost of the product and the revenue generated from its sale.
- volume of products sold, which could be used to analyze sales trends over time.
- purchasing behavior across different demographic groups
- Segmentation (M/F) and age.



## Libraries

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import pycountry
import pycountry_convert as pc
import country_converter as coco
import geopandas as gpd
from matplotlib.colors import ListedColormap
import datetime as dt

import SQL_queries as SQL_queries
import cleaning as cleaning


In [2]:
df=pd.read_csv("sales.csv")

In [3]:
df.isnull().sum()

index                   0
Date                    1
Year                    1
Month                   1
Customer Age            1
Customer Gender         1
Country                 1
State                   1
Product Category        1
Sub Category            1
Quantity                1
Unit Cost               1
Unit Price              1
Cost                    1
Revenue                 0
Column1             32293
dtype: int64

In [4]:
df=cleaning.cleaning(df)

In [16]:
df.to_csv('final_dataframe.csv', index=False)

## Exporting to sql

In [5]:
import pymysql
import sqlalchemy as alch # python -m pip install --upgrade 'sqlalchemy<2.0'

from getpass import getpass
import pandas as pd
from sqlalchemy import create_engine, text
import time


In [6]:

engine = SQL_queries.connect(df)


# Metric 1: total yearly amount of sales by sub category
## It doesn't give us useful information to make an informed decision. Let's continue

In [7]:



result= SQL_queries.yearly_sales(engine)
result

,year,sub_category,total_quantity
0,0,None,NaN
1,2016,Bike Racks,204.0
2,2015,Bike Stands,154.0
3,2016,Bike Stands,150.0
4,2016,Bottles and Cages,6216.0
5,2015,Bottles and Cages,4342.0
6,2015,Caps,1243.0
7,2016,Caps,1777.0
8,2016,Cleaners,664.0
9,2015,Cleaners,438.0


# Metric 2: % SALES amount growth from 2015 to 2016 by product
## Here we can see some interesting data. From the table below we can clearly see that there has been a sales drop for the Road Bikes sub category, Bike Stands, Mountain Bikes, etc (Bikes in general).


## In order to decide whether or no to remove these products from our portfolio we need to figure out what is the profit margin for these products, which will be explored later on.
 

In [8]:
result_1=SQL_queries.yearly_growth(engine)
result_1

,sub_category,total_quantity_2015,total_quantity_2016,sales_growth
0,Tires and Tubes,9531.0,12682.0,33.060539
1,Gloves,0.0,913.0,NaN
2,Helmets,3614.0,4773.0,32.069729
3,Bike Stands,154.0,150.0,-2.597403
4,Mountain Bikes,2762.0,2737.0,-0.905141
5,Hydration Packs,354.0,432.0,22.033898
6,Jerseys,1781.0,2252.0,26.445817
7,Fenders,0.0,1494.0,NaN
8,Cleaners,438.0,664.0,51.598174
9,Socks,317.0,433.0,36.593060


# Metric 3: Monthly sales. 

## This view is just for the company to track monthly sales 2016. Can't be used for any key decision right now


- Let's only keep the states where the count of entries is more than 20. Otherwise, the data is considered to have low reliability.




In [9]:
result_2=SQL_queries.monthly_sales(engine)
result_2

  




,sub_category,month,total_quantity
0,Bike Racks,April,29.0
1,Bike Racks,February,29.0
2,Bike Racks,January,15.0
3,Bike Racks,July,16.0
4,Bike Racks,June,33.0
5,Bike Racks,March,35.0
6,Bike Racks,May,47.0
7,Bike Stands,April,42.0
8,Bike Stands,February,15.0
9,Bike Stands,January,24.0


# Metric 4: Daily sales. 

## This view is just for the company to track daily sales 2016. It can be used to do a rough estimation of the amount of resources that on average will be needed for each week.

In [10]:
result_3=SQL_queries.total_daily_sales(engine)
result_3

    

,day_of_week,total_quantity
0,Friday,5863.0
1,Saturday,5784.0
2,Sunday,6013.0
3,Wednesday,5654.0
4,Monday,5338.0
5,Thursday,5310.0
6,Tuesday,5795.0


# Metric 5: Unitary net profit by sub category

## This is an estimation of the net profit for each product sub category. In line with our previous observation of low sales for the bike subcategory, given the low profit of this category, we will recommend the company to remove it from its portfolio.

In [11]:
result_4=SQL_queries.unitary_profit_by_sub_product(engine)
result_4


,sub_category,profit
0,Bike Racks,219.043689
1,Hydration Packs,116.787407
2,Vests,111.970577
3,Bike Stands,100.885057
4,Shorts,94.572150
5,Jerseys,90.390688
6,Helmets,74.602443
7,Fenders,60.856807
8,Gloves,59.994069
9,Touring Bikes,41.702124


# Metric 6: net profit by state

## This is an estimation of the net profit for each country. Carlifornia and England have very high profit, but some french stores don't. Let's see what's their profit margin.

In [12]:
result_6=SQL_queries.profit_by_state(engine)
result_6




,state,profit
0,California,393048.0
1,England,328955.0
2,Saarland,236515.0
3,Nordrhein-Westfalen,217394.0
4,Washington,207057.0
5,Hessen,206341.0
6,Hamburg,154458.0
7,Bayern,124348.0
8,Oregon,98262.0
9,Seine (Paris),56797.0


# Metric 7: profit margin by state

## This is an estimation of the net profit margin for each country. It measures how much a company keeps in earnings from every dollar of sales it generates. German stores have very high profit margin, but some french cities don't. We recommend closing the following stores in order to have higher total margins: Charente-Maritime, Nord and Somme.

In [ ]:
df.state.unique()

In [13]:
result_7=SQL_queries.profit_margin_state(engine)
result_7

,state,margin
0,Nordrhein-Westfalen,23.333623
1,Bayern,23.138472
2,Hessen,22.499120
3,Saarland,22.400563
4,Brandenburg,22.353513
5,Hamburg,21.631682
6,Pas de Calais,14.159215
7,Loir et Cher,12.096749
8,Val d'Oise,11.392671
9,Garonne (Haute),9.317376


# Marketing campaign. 
# Metric 8: Segmentation. 
## We can see that the average buyer age is around 36 for all product lines. However, it is difficult to make a decision out of this since we know the buyer but we don't know who are they buying it for.


## When it comes to genders, the equation is pretty balanced, meaning that our marketing campaigns shouldn't be focused on any specific gender.

In [14]:
result_8=SQL_queries.segmentation(engine)
result_8

,sub_category,average_age
0,None,0.0000
1,Bike Racks,38.6796
2,Bike Stands,36.7724
3,Bottles and Cages,36.5046
4,Caps,35.7521
5,Cleaners,37.2312
6,Fenders,37.9029
7,Gloves,37.1604
8,Helmets,36.4478
9,Hydration Packs,37.2601


In [15]:
result_9=SQL_queries.segmentation_2(engine)
result_9

,sub_category,total_women,total_men
0,None,0,0
1,Bike Racks,47,56
2,Bike Stands,80,65
3,Bottles and Cages,2591,2704
4,Caps,737,780
5,Cleaners,268,277
6,Fenders,339,423
7,Gloves,204,276
8,Helmets,2046,2130
9,Hydration Packs,192,204


# Tableau Dashboards

![title](img/pic1.png)
![title](img/pic2.png)
![title](img/pic3.png)
![title](img/pic4.png)
